In [2]:
from glmsingle.glmsingle import GLM_single
import argparse
import os
import os.path as op
from nilearn import image
from numrisk.utils.data import Subject
from nilearn.glm.first_level import make_first_level_design_matrix
import numpy as np
import pandas as pd

bids_folder = '/shares/zne.uzh/mrenke/ds-dnumrisk'

In [5]:
subject= 4

session = 1
derivatives = op.join(bids_folder, 'derivatives')

sub = Subject(subject, bids_folder=bids_folder)
base_dir = 'glm_stim1.denoise'
runs = range(1, 7)
split_data = '' # change name for ouput filename

ims = sub.get_preprocessed_bold(session=session, runs=runs)
data = [image.load_img(im).get_fdata() for im in ims]


MemoryError: 

In [6]:
onsets = sub.get_fmri_events(session=session, runs = runs) # np.shape(onsets)[0] = 2* N_trials !
tr = 2.3
n = np.shape(image.load_img(ims[0]).get_fdata())[3] # number of volumes
frametimes = np.linspace(tr/2., (n - .5)*tr, n)
onsets['onset'] = ((onsets['onset']+tr/2.) // 2.3) * 2.3

print(onsets)

dm = [make_first_level_design_matrix(frametimes, onsets.loc[run], hrf_model='fir', oversampling=100.,
                                        drift_order=0,
                                        drift_model=None).drop('constant', axis=1) for run in runs]

dm = pd.concat(dm, keys=runs, names=['run']).fillna(0) # keys = range(1, 7)
dm.columns = [c.replace('_delay_0', '') for c in dm.columns]
dm /= dm.max()
print(dm)
dm[dm < 1.0] = 0.0
print(dm.shape)

X = [dm.loc[run].values for run in runs]

print(len(X))


MemoryError: 

In [ ]:
opt = dict()

# set important fields for completeness (but these would be enabled by default)
opt['wantlibrary'] = 1
opt['wantglmdenoise'] = 1
opt['wantfracridge'] = 1

# for the purpose of this example we will keep the relevant outputs in memory
# and also save them to the disk
opt['wantfileoutputs'] = [0, 0, 0, 1]



# running python GLMsingle involves creating a GLM_single object
# and then running the procedure using the .fit() routine
glmsingle_obj = GLM_single(opt)

results_glmsingle = glmsingle_obj.fit(
    X,
    data,
    0.6,
    2.3,
    outputdir=base_dir)

betas = results_glmsingle['typed']['betasmd']
betas = image.new_img_like(ims[0], betas)
betas = image.index_img(betas, slice(None, None, 2))